In [1]:
import pandapower as pp
import pandapower.networks as pn
from pandapower.estimation import estimate, remove_bad_data, chi2_analysis
import pandas as pd
import numpy as np

print(pp.__version__)


2.0.1


Types of Measurements

* “v” for voltage measurements (in per-unit)
* “p” for active power measurements (in MW)
* “q” for reactive power measurements (in kVar)
* “i” for electrical current measurements at a line (in A)

In [29]:
net = pn.simple_four_bus_system()
pp.runpp(net)
V, delta=net.res_bus.vm_pu, net.res_bus.va_degree
# print(net)

In [30]:
df_measurement=pd.DataFrame()
df_measurement['meas_type']=['v','p','p','q','q']
df_measurement['element_type']=['bus','line','line','line','line']
df_measurement['element']=[0,0,1,0,1]
df_measurement['side']=['None','from','from','from','from']
print(df_measurement)
list_value=[]
list_std=[]
#where bus measurement has 3%error
#where line measurement has 5%error
#where trafo measurement has 5%error

for index, row in df_measurement.iterrows():
    if row['element_type'] =='bus':
        if row['meas_type'] =='v':
            mu= net.res_bus.iloc[row['element'],0]
            sigma = (abs(mu)*1.03-abs(mu)*0.97)/4
        elif row['meas_type'] =='p':
            mu= net.res_bus.iloc[row['element'],2]
            sigma = (abs(mu)*1.03-abs(mu)*0.97)/4
        elif row['meas_type'] =='q':
            mu= net.res_bus.iloc[row['element'],3]
            sigma = (abs(mu)*1.03-abs(mu)*0.97)/4

    elif row['element_type'] =='line':
        if row['side']=='from':
            if row['meas_type'] =='p':
                mu= net.res_line.iloc[row['element'],0]
                sigma = (abs(mu)*1.05-abs(mu)*0.95)/4
            elif row['meas_type'] =='q':
                mu = net.res_line.iloc[row['element'],1]
                sigma = (abs(mu)*1.05-abs(mu)*0.95)/4
            elif row['meas_type']=='i':
                mu = net.res_line.iloc[row['element'],6]
                sigma = (abs(mu)*1.05-abs(mu)*0.95)/4
        elif row['side']=='to':
            if row['meas_type'] =='p':
                mu= net.res_line.iloc[row['element'],2]
                sigma = (abs(mu)*1.05-abs(mu)*0.95)/4
            elif row['meas_type'] =='q':
                mu = net.res_line.iloc[row['element'],3]
                sigma = (abs(mu)*1.05-abs(mu)*0.95)/4
            elif row['meas_type']=='i':
                mu = net.res_line.iloc[row['element'],7]
                sigma = (abs(mu)*1.05-abs(mu)*0.95)/4
                
                
    elif row['element_type'] =='trafo':
        'create trafo measurement'
        if row['side']=='from':
            pass
        elif row['side']=='to':
            if row['meas_type'] =='v':
                mu =net.res_trafo.iloc[row['element'],10]
                sigma = (abs(mu)*1.05-abs(mu)*0.95)/4
#                 print(mu)
            elif row['meas_type'] =='p':
                mu =net.res_trafo.iloc[row['element'],2]
                sigma = (abs(mu)*1.05-abs(mu)*0.95)/4
            elif row['meas_type'] =='q':
                mu =net.res_trafo.iloc[row['element'],3]
                sigma = (abs(mu)*1.05-abs(mu)*0.95)/4
                
#     print('Mean:',mu, 'standard deviation:', sigma)
    value = np.random.normal(mu, sigma, 1)
#     print('Random value:', value[0])


    list_value.append(value[0])
    list_std.append(sigma)
df_measurement['value']=list_value
df_measurement['std_dev']=list_std


print(df_measurement)

for index, row in df_measurement.iterrows():
    if row['element_type']=='bus':
        print('create bus measurement')
        pp.create_measurement(net, row['meas_type'], row['element_type'], value=row['value'], 
                              std_dev=row['std_dev'], element=row['element'])

    elif row['element_type']=='line':
        print('create line measurement')
        pp.create_measurement(net, row['meas_type'], row['element_type'], value=row['value'], 
                              std_dev=row['std_dev'], element=row['element'], side=row['side'])
        
    elif row['element_type']=='trafo':
        print('create trafo measurement')
        if row['meas_type'] in ['p','q']:
            pp.create_measurement(net, row['meas_type'], row['element_type'], value=row['value'], 
                                  std_dev=row['std_dev'], element=row['element'], side=row['side'])

        ## why    create trafo measurement 1 meas_type v        trafo      ""to""  locate at bus 0    to  0.987147  0.024915

print(net.measurement)
success = estimate(net, init="flat", calculate_voltage_angles=True, zero_injection=[1])
# V, delta = net.res_bus_est.vm_pu, net.res_bus_est.va_degree
V_est, delta_est = net.res_bus_est.vm_pu, net.res_bus_est.va_degree
# print(net)
print(net.res_line_est)
print(net.res_line)

  meas_type element_type  element  side
0         v          bus        0  None
1         p         line        0  from
2         p         line        1  from
3         q         line        0  from
4         q         line        1  from
  meas_type element_type  element  side     value   std_dev
0         v          bus        0  None  0.983676  0.015000
1         p         line        0  from  0.027843  0.000690
2         p         line        1  from  0.015765  0.000393
3         q         line        0  from  0.013549  0.000333
4         q         line        1  from  0.007995  0.000202
create bus measurement
create line measurement
create line measurement
create line measurement
create line measurement
   name measurement_type element_type  element     value   std_dev  side
0  None                v          bus        0  0.983676  0.015000  None
1  None                p         line        0  0.027843  0.000690  from
2  None                p         line        1  0.015765  0.00

In [31]:
net.res_line_est

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,0.027843,0.013549,-0.025841,-0.013294,0.002002,0.000254,0.045597,0.04560,0.04560,0.980181,-150.216505,0.919831,-148.947641,32.112943
1,0.015765,0.007995,-0.015024,-0.007903,0.000741,0.000091,0.027737,0.02774,0.02774,0.919831,-148.947641,0.883287,-148.104530,19.535086


In [32]:
net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,0.027611,0.013328,-0.025713,-0.013088,0.001899,0.000241,0.044405,0.044408,0.044408,0.996608,-0.208127,0.93776,0.992541,31.273099
1,0.015713,0.008088,-0.015000,-0.008000,0.000713,0.000088,0.027200,0.027203,0.027203,0.937760,0.992541,0.90200,1.815913,19.157250


In [33]:
net.res_bus_est

,vm_pu,va_degree,p_mw,q_mvar
0,0.983676,0.000000,-2.847033e-02,-1.370345e-02
1,0.980181,-150.216505,-4.306620e-16,-1.876471e-15
2,0.919831,-148.947641,1.007621e-02,5.299625e-03
3,0.883287,-148.104530,1.502364e-02,7.903342e-03


In [196]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar
0,1.000000,0.000000,-0.028256,-0.013475
1,0.996608,-0.208127,0.000000,0.000000
2,0.937760,0.992541,0.010000,0.005000
3,0.902000,1.815913,0.015000,0.008000


In [197]:
V_est-V

0   -0.014629
1   -0.014711
2   -0.015933
3   -0.016356
Name: vm_pu, dtype: float64

In [198]:
delta_est-delta

0      0.000000
1   -150.007122
2   -149.951324
3   -149.895171
Name: va_degree, dtype: float64

In [23]:
net = pn.simple_four_bus_system()
net.sgen['p_mw']=[0,0]
net.sgen['q_mvar']=[0,0]
net.sgen['sn_mva']=[0,0]
pp.runpp(net)

In [27]:
df_measurement=pd.DataFrame()
# df_measurement['meas_type']=['v','v','v','v','p','p','q','q']
# df_measurement['element_type']=['bus','bus','bus','bus','line','line','line','line']
# df_measurement['element']=[0,1,2,3,0,1,0,1]
# df_measurement['side']=['None','None','None','None','from','from','from','from']
df_measurement['meas_type']=['v','v','v','v','p','q']
df_measurement['element_type']=['bus','bus','bus','bus','line','line']
df_measurement['element']=[0,1,2,3,0,0]
df_measurement['side']=['None','None','None','None','from','from']
# print(df_measurement)
list_value=[]
list_std=[]
#where bus measurement has 3%error
#where line measurement has 5%error
#where trafo measurement has 5%error
for index, row in df_measurement.iterrows():
    if row['element_type'] =='bus':
        if row['meas_type'] =='v':
            mu= net.res_bus.iloc[row['element'],0]
            sigma = (abs(mu)*1.03-abs(mu)*0.97)/4
        elif row['meas_type'] =='p':
            mu= net.res_bus.iloc[row['element'],2]
            sigma = (abs(mu)*1.03-abs(mu)*0.97)/4
        elif row['meas_type'] =='q':
            mu= net.res_bus.iloc[row['element'],3]
            sigma = (abs(mu)*1.03-abs(mu)*0.97)/4

    elif row['element_type'] =='line':
        
        if row['side']=='from':
            if row['meas_type'] =='p':
                mu= net.res_line.iloc[row['element'],0]
                sigma = (abs(mu)*1.05-abs(mu)*0.95)/4
            elif row['meas_type'] =='q':
                mu = net.res_line.iloc[row['element'],1]
                sigma = (abs(mu)*1.05-abs(mu)*0.95)/4
        elif row['side']=='to':
            pass

    elif row['element_type'] =='trafo':
        'create trafo measurement'
        if row['side']=='from':
            pass
        elif row['side']=='to':
            if row['meas_type'] =='v':
                mu =net.res_trafo.iloc[row['element'],10]
                sigma = (abs(mu)*1.05-abs(mu)*0.95)/4
#                 print(mu)
            elif row['meas_type'] =='p':
                mu =net.res_trafo.iloc[row['element'],2]
                sigma = (abs(mu)*1.05-abs(mu)*0.95)/4
            elif row['meas_type'] =='q':
                mu =net.res_trafo.iloc[row['element'],3]
#                 print(mu)
                sigma = (abs(mu)*1.05-abs(mu)*0.95)/4
#     print('Mean:',mu, 'standard deviation:', sigma)
    value = np.random.normal(mu, sigma, 1)
#     print('Random value:', value[0])


    list_value.append(value[0])
    list_std.append(sigma)
df_measurement['value']=list_value
df_measurement['std_dev']=list_std


print(df_measurement)

for index, row in df_measurement.iterrows():
    if row['element_type']=='bus':
        print('create bus measurement')
        pp.create_measurement(net, row['meas_type'], row['element_type'], value=row['value'], 
                              std_dev=row['std_dev'], element=row['element'])

    elif row['element_type']=='line':
        print('create line measurement')
        pp.create_measurement(net, row['meas_type'], row['element_type'], value=row['value'], 
                              std_dev=row['std_dev'], element=row['element'], side=row['side'])
        
    elif row['element_type']=='trafo':
        print('create trafo measurement')
        if row['meas_type'] in ['p','q']:
            pp.create_measurement(net, row['meas_type'], row['element_type'], value=row['value'], 
                                  std_dev=row['std_dev'], element=row['element'], side=row['side'])

        ## why    create trafo measurement 1 meas_type v        trafo      ""to""  locate at bus 0    to  0.987147  0.024915

print(net.measurement)
success = estimate(net, init="flat", calculate_voltage_angles=True, zero_injection=[1])
# V, delta = net.res_bus_est.vm_pu, net.res_bus_est.va_degree
V_est, delta_est = net.res_bus_est.vm_pu, net.res_bus_est.va_degree
# print(net)
print(net.res_line_est)
print(net.res_line)

  meas_type element_type  element  side     value   std_dev
0         v          bus        0  None  1.002660  0.015000
1         v          bus        1  None  0.999704  0.014892
2         v          bus        2  None  0.828950  0.012493
3         v          bus        3  None  0.754759  0.011233
4         p         line        0  from  0.077512  0.001912
5         q         line        0  from  0.022648  0.000553
create bus measurement
create bus measurement
create bus measurement
create bus measurement
create line measurement
create line measurement
   name measurement_type element_type  element     value   std_dev  side
0  None                v          bus        0  1.002660  0.015000  None
1  None                v          bus        1  0.999704  0.014892  None
2  None                v          bus        2  0.828950  0.012493  None
3  None                v          bus        3  0.754759  0.011233  None
4  None                p         line        0  0.077512  0.001912  from
5 

/Users/pornchaichaweewat/anaconda3/envs/Powersystem/lib/python3.6/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:193: MatrixRankWarning: Matrix is exactly singular
  warn("Matrix is exactly singular", MatrixRankWarning)


In [28]:
net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,0.076478,0.022123,-0.063577,-0.020459,0.012901,0.001663,0.115742,0.115744,0.115744,0.992830,-0.614985,0.832875,1.086476,81.509827
1,0.033577,0.010459,-0.030000,-0.010000,0.003577,0.000459,0.060947,0.060949,0.060949,0.832875,1.086476,0.748880,2.214442,42.921858


In [26]:
net.res_line_est

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
net.res_bus_est

,vm_pu,va_degree,p_mw,q_mvar
0,0.970038,0.000000,-0.031883,0.112355
1,0.986208,-150.611879,-0.044732,-0.137105
2,0.827412,-148.808072,0.029700,0.009672
3,0.744341,-147.506295,0.029349,0.010770


In [21]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar
0,1.000000,0.000000,-0.077385,-0.023111
1,0.992830,-0.614985,0.000000,0.000000
2,0.832875,1.086476,0.030000,0.010000
3,0.748880,2.214442,0.030000,0.010000


In [22]:
print(net.bus)

      name  vn_kv type  zone  in_service
0  bus1ref   10.0    b  None        True
1     bus2    0.4    b  None        True
2     bus3    0.4    b  None        True
3     bus4    0.4    b  None        True


In [15]:
print(net.trafo)

          name            std_type  hv_bus  lv_bus  sn_mva  vn_hv_kv  \
0  transformer  0.25 MVA 10/0.4 kV       0       1    0.25      10.0   

   vn_lv_kv  vk_percent  vkr_percent  pfe_kw  ...  tap_neutral  tap_min  \
0       0.4         4.0          1.2     0.6  ...            0       -2   

  tap_max  tap_step_percent  tap_step_degree  tap_pos  tap_phase_shifter  \
0       2               2.5              0.0        0              False   

   parallel   df  in_service  
0         1  1.0        True  

[1 rows x 23 columns]
